In [1]:
import pandas as pd, re, os, matplotlib.pyplot as plt, numpy as np, collections, itertools, tqdm, random, textwrap as tw, json
from matplotlib.markers import MarkerStyle
from matplotlib import rcParams
rcParams['font.family'] = "P052"
try:
    pass
    %load_ext autoreload
    %autoreload 2
    %config InlineBackend.figure_format = 'svg'
except SyntaxError:
    pass

os.chdir("/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/discovery/nature_atlas/")
OUR_RESULTS_FILE = "/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/out/results_2023-03-13@23`32`26.0@+00`00.csv"

In [2]:
our_results = pd.read_csv(OUR_RESULTS_FILE)
PSP = pd.read_excel("/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/data/raw_data/PSP_script_download.xlsx")

In [3]:
PSP = PSP[['KIN_ACC_ID', 'GENE', 'SUB_ACC_ID', 'SUB_GENE', 'SUB_MOD_RSD', 'SITE_+/-7_AA']]

In [4]:
PSP = PSP.applymap(lambda x: x.upper())

In [5]:
PSP['Symb'] = PSP['SUB_GENE'] + "|" + PSP['SUB_ACC_ID'] + "|" + PSP['SUB_MOD_RSD']

In [6]:
symb_to_seq = dict(zip(PSP['SUB_ACC_ID'], PSP['SITE_+/-7_AA']))

In [7]:
our_results['Site Seq'] = [symb_to_seq[eval(x)[0]] if eval(x)[0] in symb_to_seq else None for x in our_results['Site Uniprot Accession']]

In [8]:
our_results.to_csv(OUR_RESULTS_FILE.replace(".csv", "with_seq.csv"), index=False)

In [9]:
site_intersection = set(json.load(open("site_intersection.json", "r")))

In [16]:
KINU = 'ABL1|P00519'
KIN = 'ABL1'

targets = PSP[PSP['GENE'] == KIN][['KIN_ACC_ID', 'SITE_+/-7_AA']].applymap(lambda x: x.upper()).rename(columns={'KIN_ACC_ID': 'Kinase Uniprot ID', 'SITE_+/-7_AA': 'Site Sequence'}, inplace=False)

targets = targets[targets['Site Sequence'].isin(site_intersection)].sample(10, random_state=42).reset_index(drop=True)
targets['Kinase Gene Name'] = KIN
targets['Class'] = 'Target'
decoys = pd.read_csv("/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/data/raw_data/raw_data_45176_formatted_65.csv")
alls = decoys.copy()
decoys = decoys[(decoys['Gene Name of Provided Kin Seq'] == KINU) & (decoys['Class'] == 0) & (decoys['Site Sequence'].isin(site_intersection))]
print(len(decoys))
decoys = decoys[['Gene Name of Kin Corring to Provided Sub Seq', 'Site Sequence']].rename(columns={'Gene Name of Kin Corring to Provided Sub Seq':'Kinase Uniprot ID'}, inplace=False).sample(10, random_state=84).reset_index(drop=True)
decoys['Class'] = 'Decoy'
decoys['Kinase Uniprot ID'] = [x.split('|')[1] for x in decoys['Kinase Uniprot ID']]
decoys['Kinase Gene Name'] = KIN+"_D"

183


In [17]:
psp_handfull = pd.concat([targets, decoys], axis=0).reset_index(drop=True)


In [15]:
os.chdir("/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/data/raw_data")
with open('/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/discovery/nature_atlas/issue28B/sites.txt', 'w') as f:
   f.write("\n".join(psp_handfull['Site Sequence']))
with open('/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/discovery/nature_atlas/issue28B/kins.txt', 'w') as f:
   f.write(ks:=str(alls[alls['Gene Name of Kin Corring to Provided Sub Seq'] == KINU]['Kinase Sequence'].iloc[0]))
kin_info = {ks: {"Uniprot Accession ID": [KINU.split("|")[1]], "Gene Name": [KIN]}}
site_info = {ss: {"Uniprot Accession ID": ["UNK"], "Gene Name": ["Target" if i < 10 else "Decoy"], "Location": ['UNK']} for i, ss in enumerate(psp_handfull['Site Sequence'])}
with open('/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/discovery/nature_atlas/issue28B/kin-info.json', 'w') as f:
   json.dump(kin_info, f)
with open('/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/discovery/nature_atlas/issue28B/site-info.json', 'w') as f:
   json.dump(site_info, f)


In [ ]:
os.chdir("/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/data/raw_data")
with open('/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/discovery/nature_atlas/issue28B/sites.txt', 'w') as f:
   f.write("\n".join(psp_handfull['Site Sequence']))
with open('/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/discovery/nature_atlas/issue28B/kins.txt', 'w') as f:
   f.write(ks:=str(alls[alls['Gene Name of Kin Corring to Provided Sub Seq'] == KINU]['Kinase Sequence'].iloc[0]))
kin_info = {ks: {"Uniprot Accession ID": [KINU.split("|")[1]], "Gene Name": [KIN]}}
site_info = {ss: {"Uniprot Accession ID": ["UNK"], "Gene Name": ["Target" if i < 10 else "Decoy"], "Location": ['UNK']} for i, ss in enumerate(psp_handfull['Site Sequence'])}
with open('/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/discovery/nature_atlas/issue28B/kin-info.json', 'w') as f:
   json.dump(kin_info, f)
with open('/Users/druc594/Library/CloudStorage/OneDrive-PNNL/Desktop/DeepKS_/DeepKS/discovery/nature_atlas/issue28B/site-info.json', 'w') as f:
   json.dump(site_info, f)


In [81]:
our_results = our_results[our_results['Site Seq'].isin(psp_handfull['Site Sequence'])]

In [82]:
our_results

,Kinase Uniprot Accession,Site Uniprot Accession,Site Location,Kinase Gene Name,Site Gene Name,Prediction,Score,Kinase Group Prediction,Site Seq
2146,['Q9H2X6'],['Q96A00'],['S130'],['HIPK2'],['PPP1R14A'],True Phos. Pair,0.947359,CMGC,QKRHARVTVKYDRRE
4155,['Q9H2X6'],['Q01082'],['T2159'],['HIPK2'],['SPTBN1'],False Phos. Pair,0.393167,CMGC,NGATEQRTSSKESSP
4506,['Q9H2X6'],['Q01082'],['S2110'],['HIPK2'],['SPTBN1'],True Phos. Pair,0.943681,CMGC,NGATEQRTSSKESSP
4687,['Q9H2X6'],['O15297'],['S85'],['HIPK2'],['PPM1D'],True Phos. Pair,0.942159,CMGC,QPLPPRPSPAALPGG
5193,['Q9H2X6'],['Q96A00'],['T38'],['HIPK2'],['PPP1R14A'],True Phos. Pair,0.965248,CMGC,QKRHARVTVKYDRRE
...,...,...,...,...,...,...,...,...,...
75516,['Q99683'],['Q01082'],['S2110'],['ASK1'],['SPTBN1'],False Phos. Pair,0.000275,STE,NGATEQRTSSKESSP
75697,['Q99683'],['O15297'],['S85'],['ASK1'],['PPM1D'],False Phos. Pair,0.000258,STE,QPLPPRPSPAALPGG
76203,['Q99683'],['Q96A00'],['T38'],['ASK1'],['PPP1R14A'],False Phos. Pair,0.000295,STE,QKRHARVTVKYDRRE
76275,['Q99683'],['O15297'],['S54'],['ASK1'],['PPM1D'],False Phos. Pair,0.000198,STE,QPLPPRPSPAALPGG


In [70]:
our_results

,Kinase Uniprot Accession,Site Uniprot Accession,Site Location,Kinase Gene Name,Site Gene Name,Prediction,Score,Kinase Group Prediction
0,['Q9H2X6'],['P17612'],['S11'],['HIPK2'],['PRKACA'],True Phos. Pair,0.963621,CMGC
1,['Q9H2X6'],['Q9UIF9'],['S509'],['HIPK2'],['BAZ2A'],False Phos. Pair,0.222493,CMGC
2,['Q9H2X6'],['P10415'],['S87'],['HIPK2'],['BCL2'],False Phos. Pair,0.218862,CMGC
3,['Q9H2X6'],['P01100'],['S362'],['HIPK2'],['FOS'],True Phos. Pair,0.956036,CMGC
4,['Q9H2X6'],['Q92686'],['S36'],['HIPK2'],['NRGN'],True Phos. Pair,0.927217,CMGC
...,...,...,...,...,...,...,...,...
78895,['Q99683'],['Q8IWU9'],['S19'],['ASK1'],['TPH2'],False Phos. Pair,0.000066,STE
78896,['Q99683'],['O96017'],['S120'],['ASK1'],['CHEK2'],False Phos. Pair,0.000232,STE
78897,['Q99683'],['Q92831'],['S264'],['ASK1'],['KAT2B'],False Phos. Pair,0.000063,STE
78898,['Q99683'],['P55011'],['T207'],['ASK1'],['SLC12A2'],False Phos. Pair,0.000069,STE


In [48]:
orr['Ground Truth'] = orr['Site Seq'].isin(targets['Site Sequence'])

In [53]:
orr

,Kinase Uniprot Accession,Site Uniprot Accession,Site Location,Kinase Gene Name,Site Gene Name,Prediction,Score,Kinase Group Prediction,Site Seq,Ground Truth
11,['Q9H2X6'],['P16949'],['S63'],['HIPK2'],['STMN1'],True Phos. Pair,0.978125,CMGC,SVPEFPLSPPKKKDL,False
6,['Q9H2X6'],['Q9BZK7'],['S123'],['HIPK2'],['TBL1XR1'],True Phos. Pair,0.976609,CMGC,AAASQQGSAKNGENT,False
9,['Q9H2X6'],['Q9Y4G8'],['S1248'],['HIPK2'],['RAPGEF2'],True Phos. Pair,0.971170,CMGC,KKRVRRSSFLNAKKL,False
12,['Q9H2X6'],['Q9P1Z0'],['T795'],['HIPK2'],['ZBTB4'],True Phos. Pair,0.970311,CMGC,AAERPGGTPTPVIAY,True
0,['Q9H2X6'],['P17612'],['S11'],['HIPK2'],['PRKACA'],True Phos. Pair,0.963621,CMGC,EEEEIRVSINEKCGK,False
3,['Q9H2X6'],['P01100'],['S362'],['HIPK2'],['FOS'],True Phos. Pair,0.956036,CMGC,FSGFNADYEASSSRC,False
15,['Q9H2X6'],['P02730'],['T42'],['HIPK2'],['SLC4A1'],True Phos. Pair,0.945828,CMGC,AKPDSSFYKGLDLNG,False
5,['Q9H2X6'],['Q9Y6D9'],['S598'],['HIPK2'],['MAD1L1'],True Phos. Pair,0.944870,CMGC,RTKVLHMSLNPTSVA,False
18,['Q9H2X6'],['Q14511'],['T804'],['HIPK2'],['NEDD9'],True Phos. Pair,0.944159,CMGC,RNKVMNSSNQLCEQL,False
17,['Q9H2X6'],['Q13554'],['T382'],['HIPK2'],['CAMK2B'],True Phos. Pair,0.936945,CMGC,None,False


In [52]:
psp_handfull

,Kinase Uniprot ID,Site Sequence,Class,Kinase Gene Name
0,Q9H2X6,SKRVKENTPPRCAMV,Target,HIPK2
1,Q9H2X6,SSSQAISSPQRSKRV,Target,HIPK2
2,Q9H2X6,RCAMVHSSPACSTSV,Target,HIPK2
3,Q9H2X6,GFQGLQASPSKHAGY,Target,HIPK2
4,Q9H2X6,SDSGTCQSPSLEEPM,Target,HIPK2
5,Q9H2X6,PQPQHTPSPAAPPAA,Target,HIPK2
6,Q9H2X6,PPPQPQHTPSPAAPP,Target,HIPK2
7,Q9H2X6,AAERPGGTPTPVIAY,Target,HIPK2
8,Q9H2X6,AMDDLMLSPDDIEQW,Target,HIPK2
9,Q9H2X6,SGHHRAPSYSGLSMV,Target,HIPK2


In [33]:
orss = set(our_results['Site Seq'])

In [37]:
sum([x in orss for x in decoys['Site Sequence']])

10

In [101]:
psp_handfull[['Kinase Gene Name', 'Kinase Uniprot ID', 'Site Sequence', 'Class']].to_csv("psp_handfull.csv", index=False)

In [16]:
'PPPQPQHTPSPAAPP' in our_results['Site Seq']

False

In [102]:
trues.to_csv("trues.csv", index=False)

AttributeError: 'set' object has no attribute '_stat_axis_number'

In [111]:
psp_handfull['Site Sequence']

0     SKRVKENTPPRCAMV
1     SSSQAISSPQRSKRV
2     RCAMVHSSPACSTSV
3     GFQGLQASPSKHAGY
4     SDSGTCQSPSLEEPM
5     PQPQHTPSPAAPPAA
6     PPPQPQHTPSPAAPP
7     AAERPGGTPTPVIAY
8     AMDDLMLSPDDIEQW
9     SGHHRAPSYSGLSMV
10    PPPQPQHTPSPAAPP
11    LTVNKRSSPPSAGNS
12    QPLPPRPSPAALPGG
13    PGVVMASSPALPTQP
14    SPPSAGNSPSSLKFP
15    AVPEASASPKQRRSI
16    IVIPDTPSPTVSVIT
17    SKRVKENTPPRCAMV
18    PQPQHTPSPAAPPAA
19    IKKYSPPSPGVQPFG
Name: Site Sequence, dtype: object

In [105]:
our_results

,Kinase Uniprot Accession,Site Uniprot Accession,Site Location,Kinase Gene Name,Site Gene Name,Prediction,Score,Kinase Group Prediction,Site Seq
0,['Q9H2X6'],['P17612'],['S11'],['HIPK2'],['PRKACA'],True Phos. Pair,0.963621,CMGC,EEEEIRVSINEKCGK
1,['Q9H2X6'],['Q9UIF9'],['S509'],['HIPK2'],['BAZ2A'],False Phos. Pair,0.222493,CMGC,AAAFPTASPANKDVS
2,['Q9H2X6'],['P10415'],['S87'],['HIPK2'],['BCL2'],False Phos. Pair,0.218862,CMGC,RDPVARTSPLQTPAA
3,['Q9H2X6'],['P01100'],['S362'],['HIPK2'],['FOS'],True Phos. Pair,0.956036,CMGC,FSGFNADYEASSSRC
4,['Q9H2X6'],['Q92686'],['S36'],['HIPK2'],['NRGN'],True Phos. Pair,0.927217,CMGC,AAAKIQASFRGHMAR
...,...,...,...,...,...,...,...,...,...
78895,['Q99683'],['Q8IWU9'],['S19'],['ASK1'],['TPH2'],False Phos. Pair,0.000066,STE,YWARRGFSLDSAVPE
78896,['Q99683'],['O96017'],['S120'],['ASK1'],['CHEK2'],False Phos. Pair,0.000232,STE,SSLETVSTQELYSIP
78897,['Q99683'],['Q92831'],['S264'],['ASK1'],['KAT2B'],False Phos. Pair,0.000063,STE,YWHLEAPSQRRLRSP
78898,['Q99683'],['P55011'],['T207'],['ASK1'],['SLC12A2'],False Phos. Pair,0.000069,STE,TNTYYLRTFGHNTMD
